# Obter dados no drive

In [ ]:
!pip install pyxlsb

In [ ]:
from google.colab import drive
import pandas as pd
from pyxlsb import open_workbook
import random

import zipfile
import os


from PIL import Image, ImageDraw
import matplotlib.pyplot as plt

from glob import glob
from sklearn.model_selection import train_test_split

import numpy as np
import tensorflow as tf
from keras import layers, models
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [ ]:
drive.mount('/content/drive')

# Divisão de imagens e masks em amostras de treino e teste

In [ ]:
images = []
masks = []

for path in glob('/content/drive/Shared drives/Grupo T de Tech/Data/dataset_inteli/cropped_images/*/*'):
  images.append(path + '/image.tif')
  masks.append(path + '/mask.png')

In [ ]:
# Função para carregar e pré-processar uma imagem e sua máscara
def load_and_preprocess_image(image_path, mask_path, target_size):

    image = load_img(image_path, target_size=target_size)
    image = img_to_array(image) / 255.0  # Normalização entre 0 e 1

    mask = load_img(mask_path, target_size=target_size, color_mode='grayscale')
    mask = img_to_array(mask) / 255.0  # Normalização entre 0 e 1

    return image, mask

In [ ]:
with tf.device('/gpu:0'):
    # Lista para armazenar imagens e máscaras pré-processadas
    images_processed = []
    masks_processed = []

    # Carregar e pré-processar todas as imagens e máscaras
    for img_path, mask_path in zip(images, masks):
        img, mask = load_and_preprocess_image(img_path, mask_path, target_size=(256, 256))
        images_processed.append(img)
        masks_processed.append(mask)

    # Converter para arrays numpy
    images_processed = np.array(images_processed)
    masks_processed = np.array(masks_processed)


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(images_processed, masks_processed, test_size=0.2, random_state=42)

# Definição do modelo

In [ ]:
def Unet(input_shape):
    inputs = tf.keras.Input(shape=input_shape)

    # Encoder (contraction path)
    conv1 = layers.Conv2D(64, 3, activation='relu', padding='same')(inputs)
    conv1 = layers.Conv2D(64, 3, activation='relu', padding='same')(conv1)
    pool1 = layers.MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = layers.Conv2D(128, 3, activation='relu', padding='same')(pool1)
    conv2 = layers.Conv2D(128, 3, activation='relu', padding='same')(conv2)
    pool2 = layers.MaxPooling2D(pool_size=(2, 2))(conv2)

    # Decoder (expansion path)
    up3 = layers.Conv2DTranspose(64, (2, 2), strides=(2, 2), padding='same')(conv2)
    merge3 = layers.concatenate([conv1, up3], axis=3)
    conv3 = layers.Conv2D(64, 3, activation='relu', padding='same')(merge3)
    conv3 = layers.Conv2D(64, 3, activation='relu', padding='same')(conv3)

    outputs = layers.Conv2D(1, 1, activation='sigmoid')(conv3)  # Saída com um canal (máscara binária)

    model = tf.keras.Model(inputs=inputs, outputs=outputs)
    return model

In [ ]:
# Criar modelo U-Net
model = Unet(input_shape=(256, 256, 3))
model.summary()

# Treino do modelo

In [ ]:
# Compilar o modelo
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

with tf.device('/gpu:0'):
    # Treinar o modelo
    history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=20, batch_size=16)

# Avaliação do modelo

In [ ]:
# Avaliar o modelo nos dados de teste
results = model.evaluate(X_val, y_val)
print("Test Loss:", results[0])
print("Test Accuracy:", results[1])

# Prever máscaras usando o modelo
predicted_masks = model.predict(X_val)

In [ ]:
num_images_to_plot = 5  # número de imagens a serem plotadas

imgs_preditas = []

plt.figure(figsize=(15, 10))
for i in range(num_images_to_plot):
    # Plotar imagem original
    plt.subplot(num_images_to_plot, 3, i*3 + 1)
    plt.imshow(X_val[i])
    plt.title("Imagem Original")
    plt.axis('off')

    # Plotar máscara esperada (ground truth)
    plt.subplot(num_images_to_plot, 3, i*3 + 2)
    plt.imshow(y_val[i].squeeze(), cmap='gray')  # squeeze para remover o canal de cor
    plt.title("Máscara Esperada")
    plt.axis('off')

    imgs_preditas.append(np.where(predicted_masks[i].squeeze() < 0.5, 0, 1))

    # Plotar máscara gerada pelo modelo
    plt.subplot(num_images_to_plot, 3, i*3 + 3)
    plt.imshow(predicted_masks[i].squeeze(), cmap='gray')  # squeeze para remover o canal de cor
    plt.title("Máscara Gerada pelo Modelo")
    plt.axis('off')

plt.tight_layout()
plt.show()


In [ ]:
# Obter métricas de precisão e perda do histórico de treinamento
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

# Número de épocas
epochs = range(1, len(acc) + 1)

# Plotar precisão do conjunto de treino e validação
plt.plot(epochs, acc, 'r', label='Precisão do Conjunto de Treino')
plt.plot(epochs, val_acc, 'b', label='Precisão do Conjunto de Validação')
plt.title('Precisão do Conjunto de Treino e Validação')
plt.xlabel('Épocas')
plt.ylabel('Precisão')
plt.legend()
plt.show()

# Plotar perda do conjunto de treino e validação
plt.plot(epochs, loss, 'r', label='Perda do Conjunto de Treino')
plt.plot(epochs, val_loss, 'b', label='Perda do Conjunto de Validação')
plt.title('Perda do Conjunto de Treino e Validação')
plt.xlabel('Épocas')
plt.ylabel('Perda')
plt.legend()
plt.show()

In [ ]:
# Lista para armazenar os scores de IoU
iou_scores = []

# Calcular IoUs e determinar predições corretas
correct_predictions = 0
iou_threshold = 0.5

with tf.device('/gpu:0'):
    for mask, result in zip(y_val, imgs_preditas):

        intersection = np.logical_and(mask, result)
        union = np.logical_or(mask, result)

        iou_score = np.sum(intersection) / np.sum(union) if np.sum(union) != 0 else 0
        iou_scores.append(iou_score)

        # Verificar se a predição é considerada correta (IoU >= threshold)
        if iou_score >= iou_threshold:
            correct_predictions += 1

        print('IoU é: ' + str(iou_score))

# Calcular a média dos IoUs
iou_mean = np.mean(iou_scores)
print('Média dos IoU:', iou_mean)

# Calcular Coverage Ratio (CovR)
total_predictions = len(iou_scores)
covr = correct_predictions / total_predictions if total_predictions > 0 else 0
print('Coverage Ratio (CovR):', covr)